![New York City schoolbus](schoolbus.jpg)

Photo by [Jannis Lucas](https://unsplash.com/@jannis_lucas) on [Unsplash](https://unsplash.com).
<br>

Every year, American high school students take SATs, which are standardized tests intended to measure literacy, numeracy, and writing skills. There are three sections - reading, math, and writing, each with a **maximum score of 800 points**. These tests are extremely important for students and colleges, as they play a pivotal role in the admissions process.

Analyzing the performance of schools is important for a variety of stakeholders, including policy and education professionals, researchers, government, and even parents considering which school their children should attend. 

You have been provided with a dataset called `schools.csv`, which is previewed below.

You have been tasked with answering three key questions about New York City (NYC) public school SAT performance.

# 📊 NYC Schools SAT Performance Analysis

In this notebook, we will analyze NYC high school SAT data to answer the following:

1. 🧮 Which NYC schools have the **best math results** (≥ 80% of 800)?  
2. 🏆 What are the **top 10 performing schools** based on the combined SAT score?  
3. 📍 Which **borough** has the largest standard deviation in total SAT scores?  

We will use **Pandas** for data manipulation and follow a step-by-step approach.


In [1]:
# Importing libraries
import pandas as pd

# Load the dataset
schools = pd.read_csv("schools.csv")

# Preview the dataset
schools.head()


,school_name,borough,building_code,average_math,average_reading,average_writing,percent_tested
0,"New Explorations into Science, Technology and ...",Manhattan,M022,657,601,601,NaN
1,Essex Street Academy,Manhattan,M445,395,411,387,78.9
2,Lower Manhattan Arts Academy,Manhattan,M445,418,428,415,65.1
3,High School for Dual Language and Asian Studies,Manhattan,M445,613,453,463,95.9
4,Henry Street School for International Studies,Manhattan,M056,410,406,381,59.7


## Step 1: Best Math Schools
- The **maximum SAT math score** is `800`.  
- We define *best* schools as those with **average math score ≥ 80% of 800 = 640**.  
- We keep only `"school_name"` and `"average_math"` columns, and sort by `"average_math"` in descending order.


In [2]:
# Define threshold (80% of 800)
threshold = 0.8 * 800

# Filter schools with math score ≥ threshold
best_math_schools = (
    schools[schools['average_math'] >= threshold][['school_name', 'average_math']]
    .sort_values('average_math', ascending=False)
)

best_math_schools.head()


,school_name,average_math
88,Stuyvesant High School,754
170,Bronx High School of Science,714
93,Staten Island Technical High School,711
365,Queens High School for the Sciences at York Co...,701
68,"High School for Mathematics, Science, and Engi...",683


## Step 2: Top 10 Performing Schools (Total SAT Score)
We calculate the **total SAT score** as the sum of:
- `average_math`
- `average_reading`
- `average_writing`

Then we:
1. Add a new column `"total_SAT"`.  
2. Sort schools in descending order.  
3. Select the **top 10** schools.  


In [3]:
# Select necessary columns
top_schools = schools[['school_name', 'average_math', 'average_writing', 'average_reading']].copy()

# Calculate total SAT score
top_schools['total_SAT'] = top_schools[['average_math', 'average_writing', 'average_reading']].sum(axis=1)

# Drop individual SAT columns for cleaner display
top_schools.drop(columns=['average_math', 'average_writing', 'average_reading'], inplace=True)

# Sort by total_SAT in descending order
top_schools = top_schools.sort_values('total_SAT', ascending=False)

# Select top 10 schools
top_10_schools = top_schools.head(10)

top_10_schools


,school_name,total_SAT
88,Stuyvesant High School,2144
170,Bronx High School of Science,2041
93,Staten Island Technical High School,2041
174,High School of American Studies at Lehman College,2013
333,Townsend Harris High School,1981
365,Queens High School for the Sciences at York Co...,1947
5,Bard High School Early College,1914
280,Brooklyn Technical High School,1896
45,Eleanor Roosevelt High School,1889
68,"High School for Mathematics, Science, and Engi...",1889


## Step 3: Borough with Largest Standard Deviation in SAT Scores
We now identify which **borough** shows the greatest variability in combined SAT scores.

Steps:
1. Merge `borough` column with `total_SAT`.  
2. Group by borough and calculate standard deviation.  
3. Identify the borough with the **largest std deviation**.  
4. Save results in a DataFrame with:
   - `"borough"`
   - `"num_schools"` = number of schools in borough
   - `"average_SAT"` = mean total SAT
   - `"std_SAT"` = standard deviation of total SAT  

All numeric values will be **rounded to 2 decimals**.


In [4]:
# Add borough column for grouping
borough_df = top_schools.copy()
borough_df['borough'] = schools['borough']

# Group by borough and calculate std deviation
largest_std_df = (
    borough_df.groupby('borough')['total_SAT']
    .std()
    .sort_values(ascending=False)
    .reset_index()
)

# Identify borough with largest std deviation
borough = largest_std_df.loc[0, 'borough']

# Calculate required values
num_schools = schools[schools['borough'] == borough]['school_name'].nunique()
average_SAT = round(borough_df[borough_df['borough'] == borough]['total_SAT'].mean(), 2)
std_SAT = round(borough_df[borough_df['borough'] == borough]['total_SAT'].std(), 2)

# Final result DataFrame
largest_std_dev = pd.DataFrame([{
    "borough": borough,
    "num_schools": num_schools,
    "average_SAT": average_SAT,
    "std_SAT": std_SAT
}])

largest_std_dev


,borough,num_schools,average_SAT,std_SAT
0,Manhattan,89,1340.13,230.29


# ✅ Summary of Findings

1. **Best Math Schools**  
   - Schools with ≥ 640 average math score.  
   - See `best_math_schools`.  

2. **Top 10 Schools by Total SAT**  
   - Highest combined math + reading + writing scores.  
   - See `top_10_schools`.  

3. **Borough with Largest Std Deviation**  
   - The borough with most variation in performance is shown in `largest_std_dev`.  
   - It includes number of schools, mean SAT, and std deviation.  
